# Final processing script, to see the results of the classification analyses

This analyzes the data from the SVRS (which only include FaceValue,Color Value, and Value-2).

This script is quite straightforward. It loads the relevant classificaiton scores for each condition and concatenates them. It also identifies clusters that are statistically significant (after permutation testing). The results are all plotted below. 

This plot uses the output of "TemporallySpecificAnalyses_SVR_mainFunctions". You may need to adjust the "datapath" variable to point it to the correct directory.

#### Import relevant modules

In [1]:
%reload_ext autoreload
%autoreload 2
%run EEG_auxiliary_module_sptm_wICA.ipynb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
import os
import mne
import scipy as scipy
from IPython.core.debugger import set_trace
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: 40em; }</style>"))
#disable sklearn warnings
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    os.environ['PYTHONWARNINGS']='ignore'
from scipy import stats as stats
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"


C:\Users\bachmanm\AppData\Local\Temp\ipykernel_13572\2300038999.py:13: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### get average accuracy for all subjects

In [2]:
subjects = [str(s) for s in [150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,166,167,168,169,170,171,173,174,176,177,178,180,181,182,183,185,186,188,189,190,191,192,193,194,195]]

tres = .05
%matplotlib qt

    ################# get accuracy by time #####################  
fig, ax = plt.subplots(figsize=(10,10))
n_cols  = 1
n_rows  = 3

n_panel = 0
time = np.arange(-200,2000,10)
FaceV_score = np.full((len(subjects),time.shape[0]),np.nan)
ColorV_score = np.full((len(subjects),time.shape[0]),np.nan)
OverallV_score = np.full((len(subjects),time.shape[0]),np.nan)

for s,subj in enumerate(subjects):
    
    study        = 'MADeEEG1'
    studyPathInd = os.getcwd().find(study)
    dataPath     = os.path.join('Results',subj,'classification_guggenmos_noJitFix')#,,'Cluster'
    Ffile_name     = 'stimClassification_FaceValues_Stimulus_long_10ms_SVR_reref.npy'             
    Cfile_name     = 'stimClassification_ColorValues_Stimulus_long_10ms_SVR_reref.npy'             
    Vfile_name     = 'stimClassification_Value-2_Stimulus_long_10ms_SVR_reref.npy'             
    Ffname    = os.path.join(dataPath,Ffile_name)
    Cfname    = os.path.join(dataPath,Cfile_name)
    Vfname    = os.path.join(dataPath,Vfile_name)
    Fscore      = np.load(Ffname)   
    Cscore      = np.load(Cfname)    
    Vscore      = np.load(Vfname)    
    FaceV_score[s,:] = Fscore
    ColorV_score[s,:] = Cscore
    OverallV_score[s,:] = Vscore

# Run cluster based permutation
t_obs, clusters, cluster_p_values, hzero = mne.stats.spatio_temporal_cluster_1samp_test(FaceV_score,n_permutations=2**12)
good_clusters_idx = np.where(cluster_p_values < 0.05)[0]
good_clusters = [clusters[idx] for idx in good_clusters_idx]
FsigLine    = np.empty(time.shape)
FsigLine[:] = np.nan
for x in good_clusters:
    FsigLine[x] = .04

t_obs, clusters, cluster_p_values, hzero = mne.stats.spatio_temporal_cluster_1samp_test(ColorV_score,n_permutations=2**12)
good_clusters_idx = np.where(cluster_p_values < 0.05)[0]
good_clusters = [clusters[idx] for idx in good_clusters_idx]
CsigLine    = np.empty(time.shape)
CsigLine[:] = np.nan
for x in good_clusters:
    CsigLine[x] = .04
    
t_obs, clusters, cluster_p_values, hzero = mne.stats.spatio_temporal_cluster_1samp_test(OverallV_score,n_permutations=2**12)
good_clusters_idx = np.where(cluster_p_values < 0.05)[0]
good_clusters = [clusters[idx] for idx in good_clusters_idx]
VsigLine    = np.empty(time.shape)
VsigLine[:] = np.nan
for x in good_clusters:
    VsigLine[x] = .04    
# Plot FaceValue    
xticks = np.around(np.linspace(time[0],time[-1],10),decimals = 2)
timelabels = []
[timelabels.append(str(t)) for t in xticks]

ax = plt.subplot(n_rows,n_cols,1)        
# calculate average
plt.axhline(y=0,color='k',linestyle='-')
plt.xlim(-200,2000)
tick_labels = ['-200','0','200','400','600','800','1000','1200','1400','1600','1800','2000']
plt.axvline(x=1000,color='k',linestyle='--')
plt.axvline(x=0,color='k',linestyle='--')
plt.xticks(np.arange(time[0],time[-1],step=200))
plt.show
plt.plot(time,np.nanmean(FaceV_score,axis=0))
plt.plot(time, FsigLine,marker='s',  markersize=3,c='red')
plt.fill_between(time,np.nanmean(FaceV_score,axis=0) - scipy.stats.sem(FaceV_score,0), np.nanmean(OverallV_score,axis=0) + scipy.stats.sem(FaceV_score,0),alpha=.2)
plt.xlabel('time (ms)')
plt.ylabel('Fisher Z coefficient')
plt.ylim([-.03,.05])
#plt.yticks(np.arange(chance-5,chance+15,step=5))
plt.title('Face Value')

ax = plt.subplot(n_rows,n_cols,2)        
# calculate average
plt.axhline(y=0,color='k',linestyle='-')
plt.xlim(-200,2000)
tick_labels = ['-200','0','200','400','600','800','1000','1200','1400','1600','1800','2000']
plt.axvline(x=1000,color='k',linestyle='--')
plt.axvline(x=0,color='k',linestyle='--')
plt.xticks(np.arange(time[0],time[-1],step=200))
plt.show
plt.plot(time,np.nanmean(ColorV_score,axis=0))
plt.plot(time, CsigLine,marker='s',  markersize=3,c='red')
plt.fill_between(time,np.nanmean(ColorV_score,axis=0) - scipy.stats.sem(ColorV_score,0), np.nanmean(ColorV_score,axis=0) + scipy.stats.sem(ColorV_score,0),alpha=.2)
plt.xlabel('time (ms)')
plt.ylabel('Fisher Z coefficient')
plt.ylim([-.03,.05])
#plt.yticks(np.arange(chance-5,chance+15,step=5))
plt.title('Color Value')

ax = plt.subplot(n_rows,n_cols,3)        
# calculate average
plt.axhline(y=0,color='k',linestyle='-')
plt.xlim(-200,2000)
tick_labels = ['-200','0','200','400','600','800','1000','1200','1400','1600','1800','2000']
plt.axvline(x=1000,color='k',linestyle='--')
plt.axvline(x=0,color='k',linestyle='--')
plt.xticks(np.arange(time[0],time[-1],step=200))
plt.show
plt.plot(time,np.nanmean(OverallV_score,axis=0))
plt.plot(time, VsigLine,marker='s',  markersize=3,c='red')
plt.fill_between(time,np.nanmean(OverallV_score,axis=0) - scipy.stats.sem(OverallV_score,0), np.nanmean(OverallV_score,axis=0) + scipy.stats.sem(OverallV_score,0),alpha=.2)
plt.xlabel('time (ms)')
plt.ylabel('Fisher Z coefficient')
plt.ylim([-.03,.05])
#plt.yticks(np.arange(chance-5,chance+15,step=5))
plt.title('Overall Value')
        

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=.3, hspace=.8)
plt.savefig('full_set_n40_permutations_SVR_reref.eps', format='eps')




Using a threshold of 2.022691
stat_fun(H1): min=-0.925941 max=5.044694
Running initial clustering …
Found 18 clusters


  0%|          | Permuting : 0/4095 [00:00<?,       ?it/s]

Using a threshold of 2.022691
stat_fun(H1): min=-1.266080 max=5.520931
Running initial clustering …
Found 19 clusters


  0%|          | Permuting : 0/4095 [00:00<?,       ?it/s]

Using a threshold of 2.022691
stat_fun(H1): min=-1.475671 max=4.651746
Running initial clustering …
Found 16 clusters


  0%|          | Permuting : 0/4095 [00:00<?,       ?it/s]

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [3]:
# To find the significant time-clusters for each condition
time[~np.isnan(FsigLine)]

array([ 440,  450,  460,  470,  480,  490,  500,  510,  520,  530,  540,
        550,  560,  570,  580,  590,  600,  610,  620,  630,  640,  650,
        660,  670,  680,  690,  700,  710,  720,  730,  740,  750,  760,
        770,  780,  790,  800,  810,  820,  830,  840,  850,  860,  870,
        880,  890,  900,  910,  920,  930,  940,  950,  960,  970,  980,
        990, 1000, 1010, 1020, 1030, 1040, 1050, 1060, 1070, 1080, 1090,
       1100, 1110, 1120, 1130, 1140, 1150, 1160, 1170, 1180, 1190, 1200,
       1210, 1220, 1230, 1240, 1250, 1460, 1470, 1480, 1490, 1500, 1510,
       1520, 1530, 1540, 1550, 1560, 1570, 1580, 1590, 1600, 1610, 1620,
       1630, 1640, 1650, 1660, 1670, 1680, 1690, 1700, 1710, 1720, 1730])

In [4]:
time[~np.isnan(CsigLine)]

array([120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240,
       250, 260, 270, 290, 300, 310, 320, 330, 340, 350])

In [5]:
time[~np.isnan(VsigLine)]

array([ 510,  520,  530,  540,  550,  560,  570,  580,  590,  600,  610,
        620,  630,  640,  650,  660,  670,  680,  690,  700,  710,  720,
        730,  740,  750,  760,  770,  780,  790,  800,  810,  820,  830,
        840,  850,  860,  870,  880,  890,  900,  910,  920,  930,  940,
        950,  960,  970,  980,  990, 1000, 1010, 1020, 1030, 1040, 1050,
       1060, 1070, 1080, 1090, 1100, 1110, 1120, 1130, 1140, 1150, 1160,
       1170, 1180, 1190, 1200, 1210, 1220, 1230, 1240, 1250, 1260, 1270,
       1280, 1290, 1300, 1310, 1320])